##### Maestría Inteligencia de Negocios y Análisis de Datos
##### Curso: Statistical Learning I
##### Alumno: 22008375- Leonel Eduardo Contreras
##### Alumno: 22008067- Luis Pedro Perez
##### Sección L

## Proyecto Final-Modelos

#### 1. Importar librerias

In [1]:
import joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix,classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier

#### 2. Cargar Dataset

In [2]:
data=pd.read_csv('airline_satisfaction.csv')
data

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,Male,disloyal Customer,48,Business travel,Business,821,3,3,3,...,5,3,2,5,4,5,5,2,5.0,neutral or dissatisfied
1,2,Female,Loyal Customer,35,Business travel,Business,821,2,2,2,...,5,5,5,5,3,5,5,26,39.0,satisfied
2,3,Male,Loyal Customer,41,Business travel,Business,853,4,4,4,...,3,3,3,3,4,3,5,0,0.0,satisfied
3,4,Male,Loyal Customer,50,Business travel,Business,1905,2,2,2,...,5,5,5,5,3,5,4,0,0.0,satisfied
4,5,Female,Loyal Customer,49,Business travel,Business,3470,3,3,3,...,3,3,4,3,3,3,5,0,1.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,129876,Male,Loyal Customer,28,Personal Travel,Eco Plus,447,4,4,4,...,4,5,4,4,4,5,4,2,3.0,neutral or dissatisfied
129876,129877,Male,Loyal Customer,41,Personal Travel,Eco Plus,308,3,5,3,...,2,5,5,5,5,4,2,0,0.0,neutral or dissatisfied
129877,129878,Male,Loyal Customer,42,Personal Travel,Eco Plus,337,2,5,2,...,3,3,4,5,4,4,3,6,14.0,neutral or dissatisfied
129878,129879,Male,Loyal Customer,50,Personal Travel,Eco Plus,337,5,4,4,...,3,4,5,5,3,4,3,31,22.0,satisfied


#### 3. Cargar Pipeline

In [3]:
pipeline=joblib.load('pipeline_airline.pkl')

#### 4. Codificar Variable Target

In [4]:
# Conteo de varaible Target
data['satisfaction'].value_counts()

neutral or dissatisfied    73452
satisfied                  56428
Name: satisfaction, dtype: int64

In [5]:
# Codificacion variable target
ohe_target=pd.get_dummies(data['satisfaction'],drop_first=True)
data['satisfaction']=ohe_target
data['satisfaction'].value_counts()

0    73452
1    56428
Name: satisfaction, dtype: int64

#### 5. Split de datos en Train y Test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['id', 'satisfaction'], axis=1),# Varaibles X, sin el Id y sin la varaible Y
    data['satisfaction'], # Variable Y
    test_size=0.3, # Tamaño de la muestra 70% Train. 30% Test
    random_state=2023) # Aleatorio seed 2023

#### 6. Aplicar el Pipline para Ingenieria de Caracterisitcas

In [7]:
# Fit del Pipeline a las variables X
pipeline.fit(X_train, X_test)

Pipeline(steps=[('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Arrival Delay in Minutes'])),
                ('categorical_freq_encoder',
                 CountFrequencyEncoder(variables=['Class'])),
                ('one_hot_gender',
                 Mapper(mappings={'Female': 0, 'Male': 1},
                        variables=['Gender'])),
                ('one_hot_customer',
                 Mapper(mappings={'Loyal Customer': 0, 'disloyal Customer': 1},
                        variables=['Customer Type'])),
                ('one_hot_travel',
                 Mapper(mappings={'Business travel': 0, 'Personal Travel': 1},
                        variables=['Type of Travel'])),
                ('log_transform',
                 LogTransformer(variables=['Flight Distance'])),
                ('scaler', MinMaxScaler())])

In [8]:
# Transformar las variables X segun la ingenieria de caracteristicas
X_train_trans=pipeline.transform(X_train)
X_test_trans=pipeline.transform(X_test)

#### 7. Modelo Naive Bayes

In [9]:
## Instanciamos objeto para modelo
clf_NB= GaussianNB() 
clf_NB

GaussianNB()

In [10]:
# Entrenamiento de modelo
clf_NB.fit(X_train_trans,y_train)


GaussianNB()

In [11]:
# Prediccion del Modelo
y_preds_nb=clf_NB.predict(X_test_trans)
y_preds_nb

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [12]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_nb))
print("ROC AUC:",roc_auc_score(y_test,y_preds_nb))

Accuracy: 0.8598706498306129
ROC AUC: 0.8543031651217443


In [13]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_nb,rownames=["Observacion"],colnames=["Prediccion"]))

Matriz de Confusion:

 Prediccion       0      1
Observacion              
0            19799   2307
1             3153  13705


In [31]:
# Para mejorar el modelo anterior:


## Diccionario de hiper parametros
hyperparam_grid={
                 'var_smoothing':[1e-9,1e-6,1e-5]
                 }


grid_search=GridSearchCV(estimator=clf_NB,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

##Entrenar el modelo
grid_search.fit(X_train_trans,y_train)

accuracies=grid_search.cv_results_['mean_test_accuracy']
roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HiperParametros:{grid_search.best_params_}')
print(f'Mejor Metrica:{grid_search.best_score_}')
print(f'Mean Accuracy:{mean_acc}')
print(f'Mean ROC_AC:{mean_roc_aucs}')

Mejores HiperParametros:{'var_smoothing': 1e-05}
Mejor Metrica:0.9171250262590147
Mena Accuracy:0.8582245890402218
Mena ROC_AC:0.9171144766932349


#### 8. Regresion Logistica

In [15]:
## Instanciamos objeto para modelo
logit=LogisticRegression()
logit

LogisticRegression()

In [16]:
# Entrenamiento de modelo
logit.fit(X_train_trans,y_train)

LogisticRegression()

In [17]:
# Prediccion del Modelo
y_preds_logit=logit.predict(X_test_trans)
y_preds_logit

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [18]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_logit))
print("ROC AUC:",roc_auc_score(y_test,y_preds_logit))

Accuracy: 0.8727030079047325
ROC AUC: 0.8677528413691399


In [ ]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_logit,rownames=["Observacion"],colnames=["Prediccion"]))

In [20]:
# Para mejorar el modelo anterior:

## Configurar el modelo
logit=LogisticRegression(solver='saga')

## Diccionario de hiper parametros
hyperparam_grid={'penalty':['l1','l2','elasticnet'], ##l1:ridge, l2=lasso 
                 'C':[0.01,0.1,1,10], ## Lambda
                 'fit_intercept':[True, False]} 

grid_search=GridSearchCV(estimator=logit,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

##Entrenar el modelo
grid_search.fit(X_train_trans,y_train)

accuracies=grid_search.cv_results_['mean_test_accuracy']
roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HiperParametros:{grid_search.best_params_}')
print(f'Mejor Metrica:{grid_search.best_score_}')
print(f'Mean Accuracy:{mean_acc}')
print(f'Mean ROC_AC:{mean_roc_aucs}')



c:\Users\LEONEL\anaconda3\envs\PythonClass\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
160 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LEONEL\anaconda3\envs\PythonClass\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LEONEL\anaconda3\envs\PythonClass\lib\site-packages\sklearn\linear_model\_logistic.py", line 1291, in fit
    fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose, prefer=prefer)(
  File "c:\Users\LEONEL\anaconda3\envs\PythonClass\lib\site-pac

Mejores HiperParametros:{'C': 0.1, 'fit_intercept': True, 'penalty': 'l1'}
Mejor Metrica:0.92527130743196
Mena Accuracy:0.8611171186410326
Mena ROC_AC:0.9174338435458741


#### 9. Random Forest

In [10]:
## Instanciamos objeto para modelo
random_forest= RandomForestClassifier()
random_forest

RandomForestClassifier()

In [11]:
# Entrenamiento de modelo
random_forest.fit(X_train_trans,y_train)

RandomForestClassifier()

In [12]:
# Prediccion del Modelo
y_preds_forest=random_forest.predict(X_test_trans)
y_preds_forest

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [13]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_forest))
print("ROC AUC:",roc_auc_score(y_test,y_preds_forest))

Accuracy: 0.9634534442049071
ROC AUC: 0.9609122825916142


In [16]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_forest,rownames=["Observacion"],colnames=["Prediccion"]))

Matriz de Confusion:

 Prediccion       0      1
Observacion              
0            21659    447
1              977  15881


In [15]:
# Para mejorar el modelo anterior:


## Diccionario de hiper parametros
hyperparam_grid={'n_estimators':[200],
                 'criterion':['gini','entropy','log_loss']
                 }


grid_search=GridSearchCV(estimator=random_forest,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

##Entrenar el modelo
grid_search.fit(X_train_trans,y_train)

accuracies=grid_search.cv_results_['mean_test_accuracy']
roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HiperParametros:{grid_search.best_params_}')
print(f'Mejor Metrica:{grid_search.best_score_}')
print(f'Mean Accuracy:{mean_acc}')
print(f'Mean ROC_AC:{mean_roc_aucs}')

Mejores HiperParametros:{'criterion': 'log_loss', 'n_estimators': 200}
Mejor Metrica:0.9941745339769652
Mena Accuracy:0.9626579506462157
Mena ROC_AC:0.9941114543075947


#### 10. Linear Discriminant Analysis (LDA)

In [33]:
## Instanciamos objeto para modelo
LDA= LinearDiscriminantAnalysis()
LDA

LinearDiscriminantAnalysis()

In [34]:
# Entrenamiento de modelo
LDA.fit(X_train_trans,y_train)

LinearDiscriminantAnalysis()

In [35]:
# Prediccion del Modelo
y_preds_lda=LDA.predict(X_test_trans)
y_preds_lda

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [36]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_lda))
print("ROC AUC:",roc_auc_score(y_test,y_preds_lda))

Accuracy: 0.8683913355918284
ROC AUC: 0.8634600829702018


In [37]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_lda,rownames=["Observacion"],colnames=["Prediccion"]))

Matriz de Confusion:

 Prediccion       0      1
Observacion              
0            19897   2209
1             2919  13939


In [38]:
# Para mejorar el modelo anterior:


## Diccionario de hiper parametros
hyperparam_grid={
                 'solver':['svd', 'lsqr', 'eigen']
                 }


grid_search=GridSearchCV(estimator=LDA,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

##Entrenar el modelo
grid_search.fit(X_train_trans,y_train)

accuracies=grid_search.cv_results_['mean_test_accuracy']
roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HiperParametros:{grid_search.best_params_}')
print(f'Mejor Metrica:{grid_search.best_score_}')
print(f'Mean Accuracy:{mean_acc}')
print(f'Mean ROC_AC:{mean_roc_aucs}')

Mejores HiperParametros:{'solver': 'svd'}
Mejor Metrica:0.9232928263712463
Mena Accuracy:0.8693960617586963
Mena ROC_AC:0.9232928263712464


#### 11. SVM

In [9]:
## Instanciamos objeto para modelo
SVM= SVC()
SVM

SVC()

In [10]:
# Entrenamiento de modelo
SVM.fit(X_train_trans,y_train)

SVC()

In [11]:
# Prediccion del Modelo
y_preds_svm=SVM.predict(X_test_trans)
y_preds_svm

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [12]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_svm))
print("ROC AUC:",roc_auc_score(y_test,y_preds_svm))

Accuracy: 0.9499538035109332
ROC AUC: 0.947585693440068


In [13]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_svm,rownames=["Observacion"],colnames=["Prediccion"]))

Matriz de Confusion:

 Prediccion       0      1
Observacion              
0            21336    770
1             1180  15678


#### 12. XGBooot 

In [15]:
## Instanciamos objeto para modelo
xg_boost= GradientBoostingClassifier()
xg_boost

GradientBoostingClassifier()

In [16]:
# Entrenamiento de modelo
xg_boost.fit(X_train_trans,y_train)

GradientBoostingClassifier()

In [17]:
# Prediccion del Modelo
y_preds_xgboost=xg_boost.predict(X_test_trans)
y_preds_xgboost

array([1, 1, 0, ..., 1, 1, 0], dtype=uint8)

In [18]:
# Performance del Modelo
print("Accuracy:",accuracy_score(y_test,y_preds_xgboost))
print("ROC AUC:",roc_auc_score(y_test,y_preds_xgboost))

Accuracy: 0.9427676829894261
ROC AUC: 0.9406399854916621


In [19]:
# Matriz de Prediccion
print("Matriz de Confusion:\n\n",pd.crosstab(y_test,y_preds_xgboost,rownames=["Observacion"],colnames=["Prediccion"]))

Matriz de Confusion:

 Prediccion       0      1
Observacion              
0            21143    963
1             1267  15591


In [20]:
# Para mejorar el modelo anterior:


## Diccionario de hiper parametros
hyperparam_grid={
                 'loss':['log_loss', 'deviance', 'exponential'],
                 'criterion':['friedman_mse', 'squared_error']
                 }


grid_search=GridSearchCV(estimator=xg_boost,param_grid=hyperparam_grid,scoring=['roc_auc','accuracy'],refit='roc_auc',n_jobs=-1,
                         cv=KFold(n_splits=10,shuffle=True,random_state=2023)) 

##Entrenar el modelo
grid_search.fit(X_train_trans,y_train)

accuracies=grid_search.cv_results_['mean_test_accuracy']
roc_aucs=grid_search.cv_results_['mean_test_roc_auc']

mean_acc =  np.mean([x for x in accuracies if not np.isnan(x)])
mean_roc_aucs = np.mean([x for x in roc_aucs if not np.isnan(x)])

print(f'Mejores HiperParametros:{grid_search.best_params_}')
print(f'Mejor Metrica:{grid_search.best_score_}')
print(f'Mean Accuracy:{mean_acc}')
print(f'Mean ROC_AC:{mean_roc_aucs}')

Mejores HiperParametros:{'criterion': 'friedman_mse', 'loss': 'exponential'}
Mejor Metrica:0.9881415222807897
Mena Accuracy:0.94173022615719
Mena ROC_AC:0.9878301285341688


##### Elección de Modelo: Random Forest 
##### Con un ROC AC de 0.994, y los parametros: 'criterion': log_loss y n_estimators: 200